In [1]:
# import computational packages
import numpy as np
import pandas as pd 

# sparse matrix
from scipy.io import mmread

# import modeling packages
from scipy.stats import chi2, nbinom
from scipy.optimize import minimize, dual_annealing

# parallelizing
from multiprocessing import Pool

In [2]:
def get_input_and_output_vectors(spacer_sequence, proximal_genes):
    '''helper function to get input and output vectors'''
    
    # get sample of data for cells with guide
    guide_present = cell_ts_matrix.loc[spacer_sequence] == 1
    guide_cells = cell_ts_matrix.loc[spacer_sequence][guide_present]
    guide_cells = guide_cells.sample(min(len(guide_cells), 1000))
    
    # get sample of data for cells without guide
    no_guide_cells = cell_ts_matrix.loc[spacer_sequence][~guide_present]
    no_guide_cells = no_guide_cells.sample(min(len(no_guide_cells), 1000))
    
    # get merged dataframe with sampled input and output
    input_series = pd.concat([guide_cells, no_guide_cells])
    output_series = count_matrix_df.loc[proximal_genes]
    output_series = output_series[output_series.index.isin(input_series.index)]
    input_output_df = pd.merge(input_series, output_series, left_index=True, right_index=True)
    
    # get input and output vectors as np.arrays (for faster computation)
    input_vector = input_series.values
    output_vector = output_series.values
    
    return input_vector, output_vector, input_output_df.index


def adjust_index(covariate, idx):
    '''adjusts index of covariate to match input and output vectors'''
    
    index_df = pd.DataFrame(idx).set_index(0)
    covariate_df = index_df.merge(covariate, left_index=True, right_index=True)
    return covariate_df.values


def divide_by_guide(input_vector, divide_vector):
    '''divide a given divide vector based on an input vector'''
    
    # split divide vector based on guide presence
    cells_with_guide = divide_vector[input_vector.astype(bool)]
    cells_wo_guide = divide_vector[~input_vector.astype(bool)]
    
    return cells_with_guide, cells_wo_guide


def nbinom_ll(params, x, counts, s): # , s_scores, g2m_scores,
              # cell_guide_counts, percent_mito, prep_batch_1, prep_batch_2):
    '''
    this function calculates the log-likelihood for the alternative hypothesis
    alternative hypothesis - beta1 != 0 
    params[0] = beta0 (intercept term)
    params[1] = beta1 (guide effect size)
    params[2] = sqrt(disperion)
    params[3] = beta2 for cell cycle s phase score
    params[4] = beta3 for cell cycle g2m phase score
    params[5] = beta4 for cell gRNA counts
    params[6] = beta5 for percent mitochondrial transcripts
    params[7] = beta6 for prep batch 1
    params[8] = beta7 for prep batch 2
    x1 = indicator vector
    counts = vector of counts for UMIs of a gene in each cell
    s = scaling factor
    s_scores = S phase scores for each cell
    g2m_scores = G2 and M phase scores for each cell 
    cell_guide_counts = gRNA counts for each cell
    percent_mito = percentage mitochondrial transcripts for each cell 
    prep_batch_1 = indicator vector for prep batch 1 (by cell)
    prep_batch_2 = indicator vector for prep batch 2 (by cell)
    '''
    
    # calculate value for dispersion of negative binomial distribution
    disp = np.exp(params[2])
    
    # calculate mu for negative binomial distribution
    mu = np.exp(params[0] + x * params[1] # + params[3] * s_scores + 
                # params[4] * g2m_scores + params[5] * cell_guide_counts + 
                # params[6] * percent_mito + # params[7] * prep_batch_1 + 
                # params[8] * prep_batch_2 
                + np.log(s))
    
    # calculate probability value for negative binomial distribution
    prob = disp / (disp + mu)
    
    # calculate log-likelihood vector for observed counts
    ll = nbinom.logpmf(counts, n=disp, p=prob)
    
    # take sum of log likelihood vector
    return -ll.sum()


def null_ll(params, counts, s): #  s_scores, g2m_scores, 
            # cell_guide_counts, percent_mito, prep_batch_1, prep_batch_2):
    '''
    this function calculates the log-likelihood for the null hypothesis
    null hypothesis - beta1 == 0 
    params[0] = beta0 (intercept term)
    params[1] = sqrt(dispersion)
    params[2] = beta2 for cell cycle s phase score
    params[3] = beta3 for cell cycle g2m phase score
    params[4] = beta4 for cell gRNA counts
    params[5] = beta5 for percent mitochondrial transcripts
    params[6] = beta6 for prep batch 1
    params[7] = beta7 for prep batch 2
    counts = vector of counts for UMIs of a gene in each cell
    s = scaling factor
    s_scores = S phase scores for each cell
    g2m_scores = G2 and M phase scores for each cell 
    cell_guide_counts = gRNA counts for each cell
    percent_mito = percentage mitochondrial transcripts for each cell 
    prep_batch_1 = indicator vector for prep batch 1 (by cell)
    prep_batch_2 = indicator vector for prep batch 2 (by cell)
    '''
    
    # calculate value for dispersion of negative binomial distribution
    disp = np.exp(params[1])
    
    # calculate mu for negative binomial distribution 
    mu = np.exp(params[0] + # params[2] * s_scores + params[3] * g2m_scores + 
                # params[4] * cell_guide_counts + params[5] * percent_mito + 
                # params[6] * prep_batch_1 + params[7] * prep_batch_2 + 
                np.log(s))
    
    # calculate probability value for negative binomial distribution
    prob = disp / (disp + mu)
    
    # calculate log-likelihood vector for observed counts
    ll = nbinom.logpmf(counts, n=disp, p=prob)
    
    # take sum of log-likelihood vector
    return -ll.sum()


def write_success_output(spacer_sequence, proximal_genes, null, alt):
    '''write model information if both optimizations successful'''
    
    print(spacer_sequence)
    print(proximal_genes)
    
    # get null GLM information as a list
    null_coeffs = [str(x) for x in null.x]
    print(len(null_coeffs))
    print(null_coeffs)
    print('null beta 0: ' + null_coeffs[0])
    print('null dispersion: ' + null_coeffs[1])
#     print('null s score beta: ' + null_coeffs[2])
#     print('null g2m score beta: ' + null_coeffs[3])
#     print('null cell guide counts beta: ' + null_coeffs[4])
#     print('null percent mito beta: ' + null_coeffs[5])
#     print('null prep batch 1 beta: ' + null_coeffs[6])
#     print('null prep batch 2 beta: ' + null_coeffs[7])
    null_function = str(null.fun)
    print('null likelihood: ' + null_function)
    
    # get alternative GLM information as a list
    alt_coeffs = [str(x) for x in alt.x]
    print(len(alt_coeffs))
    print(alt_coeffs)
    print('alt beta 0: ' + alt_coeffs[0])
    print('alt beta 1: ' + alt_coeffs[1])
    print('alt dispersion: ' + alt_coeffs[2])
#     print('alt s score beta: ' + alt_coeffs[3])
#     print('alt g2m score beta: ' + alt_coeffs[4])
#     print('alt cell guide counts beta: ' + alt_coeffs[5])
#     print('alt percent mito beta: ' + alt_coeffs[6])
#     print('alt prep batch 1 beta: ' + alt_coeffs[7])
#     print('alt prep batch 2 beta: ' + alt_coeffs[8])
    alt_function = str(alt.fun)
    print('alt likelihood: ' + alt_function)
    
    # create list of output information and convert to comma separated string
    output_list = [proximal_genes, spacer_sequence] + null_coeffs + \
                  [null_function] + alt_coeffs + [alt_function]
    output_string = ','.join(output_list)
    # print(output_string)
    
    return True


def write_error_output(spacer_sequence, proximal_genes):
    '''write guide and gene information if one or both optimizations failed'''
    
    # create error string and write to error file 
    error_list = [proximal_genes, spacer_sequence, 'optimization failed']
    error_string = ','.join(error_list)
    print(error_string)
    
    return True


def run_likelihood_ratio_test(null_function, alt_function):
    '''helper function to run likelihood test'''
    
    # negate function outputs to get null and alternative likelihood
    null_likelihood = -1 * null_function
    alt_likelihood = -1 * alt_function
    
    # calculate test statistic and p-value
    ts = -2 * (null_likelihood - alt_likelihood)
    log_pval = chi2.logsf(ts, 1)
    pval = np.exp(log_pval)
    
    # return p-value
    return pval


def print_coeffs(x):
    print(x)


def run_glm_optimizer(spacer_sequence, proximal_genes, max_iters):
    '''function to fit null and alternative models for given guide-gene pair'''

    # get input and output vectors
    input_vector, output_vector, idx = get_input_and_output_vectors(spacer_sequence, 
                                                                    proximal_genes)
#     print(len(input_vector))
#     print(len(output_vector))
    
    # adjust covariates and scalers with new index
    scalers_arr = adjust_index(scaling_factors, idx)
#     cell_s_arr = adjust_index(cell_s_scores, idx)
#     cell_g2m_arr = adjust_index(cell_g2m_scores, idx)
#     cell_guide_arr = adjust_index(cell_guide_counts, idx)
#     percent_mito_arr = adjust_index(percent_mito, idx)
#     prep_batch_1_arr = adjust_index(prep_batch_1, idx)
#     prep_batch_2_arr = adjust_index(prep_batch_2, idx)

    # get subsets of output vector based on guide presence in cells
    cells_with_guide, cells_wo_guide = divide_by_guide(input_vector, 
                                                       output_vector)
    
    # get subsets of scaling factors based on guide presence in cells
    scalers_with_guide, scalers_wo_guide = divide_by_guide(input_vector, 
                                                            scalers_arr)

    # calculate initial parameter estimates
    beta0_estimate = np.mean(np.log(cells_wo_guide + 1) - \
                             np.log(scalers_wo_guide))
    # beta0_estimate = 1
#     beta0_estimate = np.random.random() * 20
    beta1_estimate = beta0_estimate - np.mean(np.log(cells_with_guide + 1) - \
                                             np.log(scalers_with_guide))
    disp_estimate = -1
    
#     print(beta0_estimate)
#     print(disp_estimate)
    
    model_diff = np.Inf
    null_estimates = (beta0_estimate, disp_estimate)
    
    # fit null model using scipy optimizer
    null = minimize(null_ll, 
                    x0=null_estimates, # 0, 0, 0, 0, 0, 0), 
                    args=(output_vector, scalers_arr), 
                          # cell_s_arr, cell_g2m_arr,
                          # cell_guide_arr, percent_mito_arr,
                          # prep_batch_1_arr, prep_batch_2_arr),
                    method='Nelder-Mead', 
                    options={'maxiter': max_iters})
                    # callback=print_coeffs) # 'maxfev': max_iters,
                             # 'xatol': 1e-5, 'fatol': 1e-5})
        
    alt_estimates = (beta0_estimate, beta1_estimate, disp_estimate)
    
    # fit alternative model using scipy optimizer
    alt = minimize(nbinom_ll, 
                   x0=alt_estimates, # 0, 0, 0, 0, 0, 0), 
                   args=(input_vector, output_vector, 
                         scalers_arr), # , cell_s_arr, cell_g2m_arr,
                         # cell_guide_arr, percent_mito_arr,
                         # prep_batch_1_arr, prep_batch_2_arr),
                   method='Nelder-Mead', 
                   options={'maxiter': max_iters})
                   # callback=print_coeffs) # 'maxfev': max_iters,
                            # 'xatol': 1e-5, 'fatol': 1e-5})
    
#     while model_diff > 70:

#         # fit null model using scipy optimizer
#         null = minimize(null_ll, 
#                         x0=null_estimates, # 0, 0, 0, 0, 0, 0), 
#                         args=(output_vector, scalers_arr), 
#                               # cell_s_arr, cell_g2m_arr,
#                               # cell_guide_arr, percent_mito_arr,
#                               # prep_batch_1_arr, prep_batch_2_arr),
#                         method='Nelder-Mead', 
#                         options={'maxiter': max_iters},
#                         callback=print_coeffs) # 'maxfev': max_iters,
#                                  # 'xatol': 1e-5, 'fatol': 1e-5})

#         # fit alternative model using scipy optimizer
#         alt = minimize(nbinom_ll, 
#                        x0=alt_estimates, # 0, 0, 0, 0, 0, 0), 
#                        args=(input_vector, output_vector, 
#                              scalers_arr), # , cell_s_arr, cell_g2m_arr,
#                              # cell_guide_arr, percent_mito_arr,
#                              # prep_batch_1_arr, prep_batch_2_arr),
#                        method='Nelder-Mead', 
#                        options={'maxiter': max_iters},
#                        callback=print_coeffs) # 'maxfev': max_iters,
#                                 # 'xatol': 1e-5, 'fatol': 1e-5})
        
#         model_diff = np.abs(null.fun - alt.fun)
#         print(model_diff)
        
#         if null.fun < alt.fun:
#             null_estimates = (null.x[0], null.x[1])
#             alt_estimates = (null.x[0], 0, null.x[1])
        
#         if alt.fun < null.fun:
#             null_estimates = (alt.x[0], alt.x[2])
#             alt_estimates = (alt.x[0], alt.x[1], alt.x[2])

#     null_bounds = ((0, 20), (0, 5), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2))
#     alt_bounds = ((0, 20), (-2, 2), (0, 5), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2))
#     null = dual_annealing(null_ll, x0=(beta0_estimate, disp_estimate, 0, 0, 0, 0, 0, 0),
#                           bounds=null_bounds,
#                           args=(output_vector, scalers_arr, cell_s_arr, cell_g2m_arr, 
#                                 cell_guide_arr, percent_mito_arr, prep_batch_1_arr,
#                                 prep_batch_2_arr),
#                           maxiter=max_iters)
    
#     alt = dual_annealing(nbinom_ll, x0=(beta0_estimate, 0, disp_estimate, 0, 0, 0, 0, 0, 0),
#                          bounds=alt_bounds,
#                          args=(output_vector, scalers_arr, cell_s_arr, cell_g2m_arr, 
#                                cell_guide_arr, percent_mito_arr, prep_batch_1_arr,
#                                prep_batch_2_arr),
#                          maxiter=max_iters) 
    
    # if both optimizations run successfully
    if null.success and alt.success:
        
        # print successful output
        print('successful output!')
        
        # write model information to results file
        write_success_output(spacer_sequence, proximal_genes, null, alt)
        
        fold_change_expression = np.exp(alt.x[0] + alt.x[1]) / np.exp(null.x[0])
        print('fold change expression: ' + str(fold_change_expression))
        
        # perform likelihood ratio test and return p-value
        # pval = run_likelihood_ratio_test(null.fun, alt.fun)
        pval = run_likelihood_ratio_test(null.fun, alt.fun)
        
#         null_beta0 = null.x[0]
#         null_disp = null.x[1]
        
#         alt_beta0 = alt.x[0]
#         alt_beta1 = alt.x[1]
#         alt_disp = alt.x[2]
        
# #         print(alt_beta0)
# #         print(alt_beta1)
# #         print(alt_disp)
        
#         disp_values = np.arange(-1.5, 1, 0.1)
#         beta1_values = np.arange(-1, 1, 0.5)
#         beta0_values = np.arange(8.8, 9.3, 0.05)
        
#         null_lls = []
#         alt_lls = []
        
#         ll_values = [0] * len(disp_values)
#         counter = 0
        
#         for disp in disp_values:
#             null_lls.append(null_ll_disp(null_beta0, disp, output_vector, scalers_arr))
#             alt_lls.append(nbinom_ll_disp(alt_beta0, alt_beta1, disp, 
#                                           output_vector, scalers_arr, input_vector))
            
#         for disp in disp_values:
#             disp_lls = []
#             for beta0 in beta0_values:
#                 disp_lls.append(null_ll_disp(beta0, disp, output_vector, scalers_arr))
#             ll_values[counter] = disp_lls
#             counter += 1
            
#             alt_lls.append(nbinom_ll_disp(alt_beta0, alt_beta1, value, 
#                                           output_vector, scalers_arr, input_vector))
#         fig = plt.figure()
#         ax = plt.gca()
#         # print(ll_values)
#         plt.contourf(beta0_values, disp_values, ll_values)
#         fig, axs = plt.subplots(2)
#         axs[0].plot(disp_values, null_lls)
#         axs[1].plot(disp_values, alt_lls)
        
        return pval
    
    
    # if one or both of the optimizations fail
    else:
        
        # print error output
        print('error output')
        print(null.success)
        print(alt.success)
        
        # write guide-gene info to error file
        write_error_output(spacer_sequence, proximal_genes)
        
        # return np.nan as a placeholder
        return np.nan

In [4]:
# set path to project directory 
project_path = '/iblm/netapp/home/karthik/gasperini_project/'
data_path = '/iblm/netapp/data1/external/Gasperini2019/'

# read in cell-guide matrix
print('loading in cell-guide matrix...')
cell_ts_matrix = pd.read_hdf(project_path + 'data/cell_ts_matrix.h5')
cell_ts_matrix.index = cell_ts_matrix.index.str.lower()

# read in scaling factors
print('reading in scaling factors...')
scaling_factors = pd.read_csv(project_path + 'data/scaling_factors.csv')
scaling_factors.columns = ['cell', 'scaling_factor']
scaling_factors = scaling_factors.set_index('cell')

# filter scaling factors to only include cells in cell-guide matrix
print('creating scaling factors array (for computation)...')
scaling_factors = scaling_factors.merge(cell_ts_matrix.iloc[0], on='cell')
scaling_factors = scaling_factors.iloc[:, 0]
scaling_factors_arr = scaling_factors.values # np.array for faster computation
scalers_arr = scaling_factors_arr

# read in UMI count matrix
print('reading in count matrix...')
count_matrix = mmread(data_path + 'suppl/GSE120861_at_scale_screen.exprs.mtx')
count_matrix_df = pd.DataFrame(count_matrix.toarray())

loading in cell-guide matrix...
reading in scaling factors...
creating scaling factors array (for computation)...
reading in count matrix...


In [5]:
# read in column names from corresponding cells file
colnames = []
with open(data_path + 'suppl/GSE120861_at_scale_screen.cells.txt') as f:
    colnames = f.readlines()
colnames = pd.Series(colnames).str.strip()

# read in index (row names) from corresponding genes file 
rownames = []
with open(data_path + 'suppl/GSE120861_at_scale_screen.genes.txt') as f:
    rownames = f.readlines()
rownames = pd.Series(rownames).str.strip()

# assign row and column names to UMI count matrix
count_matrix_df.index = rownames
count_matrix_df.columns = colnames

In [6]:
# filter for top half of cells
scaling_factors = scaling_factors.sort_values(ascending=False).iloc[0:len(scaling_factors) // 2]
scaling_factors_arr = scaling_factors.values
scalers_arr = scaling_factors_arr

count_matrix_df = (count_matrix_df.T[count_matrix_df.columns.isin(scaling_factors.index)]).T
cell_ts_matrix = (cell_ts_matrix.T[cell_ts_matrix.columns.isin(scaling_factors.index)]).T

In [7]:
# read in cell cycle scores and merge into single dataframe
print('reading in cell cycle scores...')
s_scores = pd.read_csv(project_path + 'data/s_scores.csv', index_col=0)
g2m_scores = pd.read_csv(project_path + 'data/g2m_scores.csv', index_col=0)
s_scores.index = s_scores.index.rename('cell')
g2m_scores.index = g2m_scores.index.rename('cell')
merged_scores = pd.merge(s_scores, g2m_scores, on='cell')

# filter scores to only include cells in guide matrix
in_guide_matrix = merged_scores.index.isin(cell_ts_matrix.columns)
merged_scores = merged_scores[in_guide_matrix]
cell_s_scores = merged_scores['S.Score']
cell_g2m_scores = merged_scores['G2M.Score']

reading in cell cycle scores...


In [8]:
# read in phenodata file to add additional covariates
print('adding Gasperini paper covariates')
colnames = open(data_path + 'suppl/GSE120861_at_scale.phenoData.colnames.txt')\
           .read().splitlines()
phenodata_df = pd.read_csv(project_path + 'data/phenodata.txt', sep=' ', 
                           names=colnames)
phenodata_df = phenodata_df.dropna().reset_index(drop=True)

# get guide count, mitochondrial transcripts, and prep batch as covariate
cell_guide_counts = phenodata_df['guide_count']
percent_mito = phenodata_df['percent.mito']
prep_batch_1 = (phenodata_df['prep_batch'] == 'prep_batch_1').astype(np.int64)
prep_batch_2 = (phenodata_df['prep_batch'] == 'prep_batch_2').astype(np.int64)

# cell_guide_counts.index = cell_ts_matrix.columns
# percent_mito.index = cell_ts_matrix.columns
# prep_batch_1.index = cell_ts_matrix.columns
# prep_batch_2.index = cell_ts_matrix.columns

adding Gasperini paper covariates


In [9]:
# read in guide-gene pairs data frame and drop genes not in genes data frame
print('reading in guide-gene pairs...')
guide_gene_pairs = pd.read_csv(project_path + 
                               'data/high_confidence_ts_genes.csv')
guide_gene_pairs = guide_gene_pairs[guide_gene_pairs['high_confidence_subset']]
guide_gene_pairs = guide_gene_pairs[['Target_Site', 'ENSG']]
guide_gene_pairs.columns = ['target_site', 'gene']
guide_gene_pairs['target_site'] = guide_gene_pairs['target_site'].apply(lambda x: x + '_top_two').str.lower()
in_genes_df = guide_gene_pairs['gene'].isin(count_matrix_df.index)
guide_gene_pairs = guide_gene_pairs[in_genes_df].reset_index(drop=True)
guide_gene_pairs['target_site'] = guide_gene_pairs['target_site'].str.lower()
in_cell_guide_mtx = guide_gene_pairs['target_site'].isin(cell_ts_matrix.index)
guide_gene_pairs = guide_gene_pairs[in_cell_guide_mtx].reset_index(drop=True)

reading in guide-gene pairs...


In [10]:
# write guide gene pairs to CSV file 
guide_gene_pairs.to_csv(project_path + 'results/high_confidence_validations.csv', 
                        index=False)

In [15]:
guide_gene_pairs = guide_gene_pairs.sample(10)

In [16]:
# format np array to run optimizer
optimizer_df = guide_gene_pairs[['target_site', 'gene']]
optimizer_df = optimizer_df.to_numpy()

In [17]:
iter_optimizer_df = []
for lst in optimizer_df:
    iter_optimizer_df.append(np.append(lst, 5000))
iter_optimizer_df = np.array(iter_optimizer_df)

In [18]:
len(iter_optimizer_df)

10

In [19]:
iter_optimizer_df

array([['chr7.3907_top_two', 'ENSG00000189056', 5000],
       ['chr10.143_top_two', 'ENSG00000067082', 5000],
       ['chr1.10033_top_two', 'ENSG00000116199', 5000],
       ['chr15.1149_top_two', 'ENSG00000128872', 5000],
       ['chr4.1828_top_two', 'ENSG00000124882', 5000],
       ['chr1.7885_top_two', 'ENSG00000081026', 5000],
       ['chr4.2004_top_two', 'ENSG00000163297', 5000],
       ['chr9.120_top_two', 'ENSG00000099219', 5000],
       ['chr22.1058_top_two', 'ENSG00000128294', 5000],
       ['chr10.3436_top_two', 'ENSG00000107438', 5000]], dtype=object)

In [20]:
# create multiprocessing pool of 100 cores and run in parallel
pool = Pool(10)
p_values = pool.starmap(run_glm_optimizer, iter_optimizer_df)
pool.close()
pool.join()

successful output!
chr22.1058_top_two
ENSG00000128294
2
['12.144379253122995', '0.8494520289102145']
null beta 0: 12.144379253122995
null dispersion: 0.8494520289102145
null likelihood: 2328741.625218029
3
['12.153641136914668', '-0.048694098143447814', '0.8504640355341097']
alt beta 0: 12.153641136914668
alt beta 1: -0.048694098143447814
alt dispersion: 0.8504640355341097
alt likelihood: 2328520.6405782476
fold change expression: 0.9613351165060653
successful output!
chr7.3907_top_two
ENSG00000189056
2
['10.865598069614034', '-0.26757256349639125']
null beta 0: 10.865598069614034
null dispersion: -0.26757256349639125
null likelihood: 1339088.541234595
3
['10.93276674121881', '-0.31126239912510395', '-0.2447907835654881']
alt beta 0: 10.93276674121881
alt beta 1: -0.31126239912510395
alt dispersion: -0.2447907835654881
alt likelihood: 1335789.9320308312
fold change expression: 0.7834142033509979
successful output!
chr9.120_top_two
ENSG00000099219
2
['9.221786936550558', '-0.79892577013

In [21]:
p_values

[0.0,
 0.0,
 2.0711129505047859e-252,
 5.477650350834048e-88,
 0.0,
 1.7684812937456084e-166,
 0.0,
 0.0,
 4.0351272918629966e-98,
 1.3362992827331772e-253]